In [130]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [131]:
import MySQLdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [132]:
# データベース接続設定
db_config = {
    'host': 'localhost',
    'user': '***',
    'passwd': '', 
    'db': 'vhoi5_aplus_tsukuba'
}

In [133]:
# データベースに接続
conn = MySQLdb.connect(**db_config)
cursor = conn.cursor()

In [134]:
def execute_query(query):
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    return pd.DataFrame(cursor.fetchall(), columns=columns)

In [135]:
# 1. データベース内のテーブル一覧を取得
tables_query = """
SHOW TABLES
"""
tables_df = execute_query(tables_query)
print("データベース内のテーブル:")
print(tables_df)


データベース内のテーブル:
   Tables_in_vhoi5_aplus_tsukuba
0                     auth_group
1         auth_group_permissions
2                auth_permission
3                      auth_user
4               auth_user_groups
5     auth_user_user_permissions
6                   board_notice
7                     board_post
8                    board_reply
9                   board_review
10             board_review_tags
11                 board_subject
12                     board_tag
13               board_textbooks
14                  board_thread
15              django_admin_log
16           django_content_type
17             django_migrations
18                django_session


In [136]:
thread_structure_query = """
DESCRIBE board_subject
"""
thread_structure = execute_query(thread_structure_query)
print("board_subjectテーブルの構造:")
print(thread_structure)

board_subjectテーブルの構造:
          Field           Type Null  Key Default           Extra
0   internal_id        int(11)   NO  PRI    None  auto_increment
1          code   varchar(128)  YES         None                
2          name  varchar(4096)  YES         None                
3      teachers  varchar(4096)  YES         None                
4       subtype  varchar(1024)  YES         None                
5       schools  varchar(1024)  YES         None                
6      colleges  varchar(1024)  YES         None                
7  thread_id_id     bigint(20)   NO  MUL    None                
8          year        int(11)  YES         None                


In [137]:
# board_threadテーブルの行数を取得
thread_count_query = """
SELECT COUNT(*) as row_count FROM board_subject
"""
row_count = execute_query(thread_count_query).iloc[0]['row_count']
print(f"\nテーブルの総行数: {row_count}")



テーブルの総行数: 7927


In [138]:
# コードの先頭文字を取得し、グループに分類し、対応するthread_id_idを取得するクエリ
code_thread_query = """
SELECT 
    CASE 
        WHEN LEFT(code, 1) REGEXP '[0-9]' THEN '0-9'
        WHEN LEFT(code, 1) = 'A' THEN 'A'
        WHEN LEFT(code, 1) = 'B' THEN 'B'
        WHEN LEFT(code, 1) = 'C' THEN 'C'
        WHEN LEFT(code, 1) = 'E' THEN 'E'
        WHEN LEFT(code, 1) = 'F' THEN 'F'
        WHEN LEFT(code, 1) = 'G' THEN 'G'
        WHEN LEFT(code, 1) = 'H' THEN 'H'
        WHEN LEFT(code, 1) = 'W' THEN 'W'
        WHEN LEFT(code, 1) = 'Y' THEN 'Y'
        ELSE 'その他'
    END AS code_group,
    thread_id_id
FROM board_subject
"""

In [139]:
results = execute_query(code_thread_query)

In [140]:
# 各グループごとにユニークなthread_id_idのリストを作成
thread_id_lists = {}
for group in ['0-9', 'A', 'B', 'C','E', 'F', 'G', 'H', 'W', 'Y', 'その他']:
    thread_id_lists[group] = list(set(results[results['code_group'] == group]['thread_id_id']))

In [148]:
thread_id_lists['その他']

[5632,
 5633,
 5634,
 5635,
 5636,
 5637,
 5638,
 5639,
 5640,
 5641,
 5642,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5649,
 5650,
 5651,
 5652,
 5653,
 5654,
 5655,
 5656,
 5657,
 5658,
 5659,
 5660,
 5661,
 5662,
 5663,
 5664,
 5665,
 5666,
 5667,
 5668,
 5669,
 5670,
 5671,
 5672,
 5673,
 5674,
 5675,
 5676,
 5677,
 5678,
 5679,
 5680,
 5681,
 5682,
 5683,
 5684,
 5685,
 5686,
 5687,
 5688,
 5689,
 5690,
 5691,
 5692,
 5693,
 5694,
 5695,
 5696,
 5697,
 5698,
 5699,
 5700,
 5701,
 5702,
 5703,
 5704,
 5705,
 5706,
 5707,
 5708,
 5709,
 5710,
 5711,
 5712,
 5201,
 5202,
 5203,
 5204,
 5205,
 5206,
 5207,
 5208,
 5209,
 5210,
 5211,
 5212,
 5213,
 5214,
 5215,
 5216,
 5217,
 5717,
 5718,
 5723,
 5724,
 5719,
 5726,
 5727,
 5728,
 5729,
 5720,
 5731,
 5732,
 5733,
 5734,
 5721,
 5736,
 5737,
 5738,
 5739,
 5722,
 5741,
 5742,
 5743,
 5744,
 5745,
 5746,
 5747,
 5748,
 5749,
 5750,
 5751,
 5752,
 5753,
 5754,
 5725,
 5756,
 5757,
 5758,
 5759,
 5760,
 5761,
 5762,
 5763,
 5764,
 5765,

In [145]:
len(thread_id_lists['その他'])

296

In [143]:
all_count = 0
for group, thread_ids in thread_id_lists.items():
    print(f"{group}: {len(thread_ids)}")
    all_count += len(thread_ids)

print(all_count)

0-9: 1389
A: 1181
B: 472
C: 340
E: 704
F: 826
G: 363
H: 301
W: 302
Y: 579
その他: 296
6753


In [150]:
# 出力ファイルの名前
output_file = 'thread_id_lists_output.txt'

# ファイルに書き込む
with open(output_file, 'w') as f:
    for group, thread_ids in thread_id_lists.items():
        # グループ名を書き込む
        f.write(f"{group}:\n")
        
        # thread_id_idをソートし、カンマで区切って横に並べる
        sorted_ids = sorted(thread_ids)
        f.write(', '.join(map(str, sorted_ids)))
        
        # 次のグループのために2行空ける
        f.write('\n\n')

print(f"complete output file name: {output_file}")

complete output file name: thread_id_lists_output.txt


In [144]:
# # データのサンプルを取得
# sample_query = """
# SELECT * FROM board_subject
# LIMIT 7100
# """
# sample_data = execute_query(sample_query)
# print("\nboard_threadテーブルのデータサンプル:")
# print(sample_data)


board_threadテーブルのデータサンプル:
      internal_id     code          name           teachers      subtype  \
0               1  1101102  ファーストイヤーセミナー              西村 雄太         総合科目   
1               2  1101202  ファーストイヤーセミナー              清水 克志         総合科目   
2               3  1101302  ファーストイヤーセミナー              中村 友香         総合科目   
3               4  1101402  ファーストイヤーセミナー              和田 尚明         総合科目   
4               5  1101502  ファーストイヤーセミナー              小泉 利恵         総合科目   
...           ...      ...           ...                ...          ...   
7095         7096  YBF4922      彫塑特別演習ⅡB  大原 央聡,宮坂 慎司,川島 史也  専門基礎科目・専門科目   
7096         7097  YBF9017       彫塑領域研究I  大原 央聡,宮坂 慎司,川島 史也  専門基礎科目・専門科目   
7097         7098  YBF9027      彫塑領域研究II  大原 央聡,宮坂 慎司,川島 史也  専門基礎科目・専門科目   
7098         7099  YBF9037     彫塑領域特別演習I  大原 央聡,宮坂 慎司,川島 史也  専門基礎科目・専門科目   
7099         7100  YBF9047    彫塑領域特別演習II  大原 央聡,宮坂 慎司,川島 史也  専門基礎科目・専門科目   

           schools    colleges  thread_id_id  year  
0     フ